<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [189]:
import folium
import pandas as pd

In [190]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [203]:
## Task 1: Mark all launch sites on a map
latitude = spacex_df['Lat'].mean()
longitude = spacex_df['Long'].mean()
spacex_map = folium.Map(location=[latitude, longitude], zoom_start=6)

launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

for index, site in launch_sites.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=site['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket')
    ).add_to(spacex_map)

# Save the map as an HTML file
spacex_map.save("task1_launch_sites_map.html")
print("Task 1 map saved as 'task1_launch_sites_map.html'")

Task 1 map saved as 'task1_launch_sites_map.html'


## Interactive Map For launch Site:
- [Launch Site Map1](https://github.com/ShaWritesCode/MissionSpaceX_A-DataScience-Approach/blob/main/task1_launch_sites_map.html)

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [206]:
# Download and read the `spacex_launch_geo.csv`

# Load the dataset
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(URL)

print(spacex_df.head())
print(spacex_df.info())

   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

Now, you can take a look at what are the coordinates for each site.


In [208]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [211]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [213]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [221]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [225]:
# Task 2: Mark the success/failed launches for each site on the map
for index, launch in spacex_df.iterrows():
    # Use green for success and red for failure
    marker_color = 'green' if launch['class'] == 1 else 'red'
    folium.CircleMarker(
        location=[launch['Lat'], launch['Long']],
        radius=5,
        color=marker_color,
        fill=True,
        fill_opacity=0.7,
        popup=f"Site: {launch['Launch Site']}<br>Result: {'Success' if launch['class'] == 1 else 'Failure'}"
    ).add_to(spacex_map)

# Save the updated map
spacex_map.save("task2_success_failure_map.html")
print("Task 2 map saved as 'task2_success_failure_map.html'")

Task 2 map saved as 'task2_success_failure_map.html'


## Interactive Map for Success / Failure Launches:
- [Success Failure Map 2](https://github.com/ShaWritesCode/MissionSpaceX_A-DataScience-Approach/blob/main/task2_success_failure_map.html)

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [227]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [231]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [233]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Create a map centered at the average latitude and longitude
from folium.plugins import MarkerCluster

# Check if 'class' column exists
if 'class' not in spacex_df.columns:
    raise ValueError("'class' column is missing from the DataFrame!")

# Create 'marker_color' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Verify the creation of the 'marker_color' column
print(spacex_df[['Launch Site', 'class', 'marker_color']].head())

# Create a map centered at the average latitude and longitude
latitude = spacex_df['Lat'].mean()
longitude = spacex_df['Long'].mean()
spacex_map = folium.Map(location=[latitude, longitude], zoom_start=6)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(spacex_map)

# Add markers for each launch record to the cluster
for index, launch in spacex_df.iterrows():
    folium.Marker(
        location=[launch['Lat'], launch['Long']],
        popup=f"Site: {launch['Launch Site']}<br>Class: {'Success' if launch['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=launch['marker_color'], icon='info-sign')
    ).add_to(marker_cluster)

   Launch Site  class marker_color
0  CCAFS LC-40      0          red
1  CCAFS LC-40      0          red
2  CCAFS LC-40      0          red
3  CCAFS LC-40      0          red
4  CCAFS LC-40      0          red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [235]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

## Interactive Map For Marker Cluster:
- [Marker Cluster Map](https://github.com/ShaWritesCode/MissionSpaceX_A-DataScience-Approach/blob/main/task_marker_cluster_map.html)

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [240]:
# TASK 3: Calculate the distances between a launch site to its proximities
!conda install -c conda-forge geopy -y
from geopy.distance import geodesic

# Define the coordinates of the launch sites
launch_sites = {
    'CCAFS LC-40': (28.562302, -80.577356),
    'VAFB SLC-4E': (34.632834, -120.610746),
    'KSC LC-39A': (28.573255, -80.646895),
    'CCAFS SLC-40': (28.563197, -80.576820)
}

# Function to calculate distance between a launch point and a given site
def calculate_geodesic_distance(row, site_coords):
    launch_coords = (row['Lat'], row['Long'])
    return geodesic(launch_coords, site_coords).km

# Calculate distances to all launch sites and store in new columns
for site, coords in launch_sites.items():
    spacex_df[site] = spacex_df.apply(calculate_geodesic_distance, axis=1, site_coords=coords)

# Find the nearest launch site for each record
spacex_df['Nearest Site'] = spacex_df[launch_sites.keys()].idxmin(axis=1)
spacex_df['Distance to Nearest Site (km)'] = spacex_df[launch_sites.keys()].min(axis=1)

# Display the updated DataFrame
print(spacex_df[['Lat', 'Long', 'Nearest Site', 'Distance to Nearest Site (km)']].head())

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.

         Lat       Long Nearest Site  Distance to Nearest Site (km)
0  28.562302 -80.577356  CCAFS LC-40                       0.000047
1  28.562302 -80.577356  CCAFS LC-40                       0.000047
2  28.562302 -80.577356  CCAFS LC-40                       0.000047
3  28.562302 -80.577356  CCAFS LC-40                       0.000047
4  28.562302 -80.577356  CCAFS LC-40                       0.000047


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [243]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [246]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [248]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
from geopy.distance import geodesic
import folium

launch_site_lat = 28.56319  
launch_site_lon = -80.57682  

# Coordinates of the coastline 
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance between the launch site and the coastline
launch_site_coords = (launch_site_lat, launch_site_lon)
coastline_coords = (coastline_lat, coastline_lon)

distance_to_coastline = geodesic(launch_site_coords, coastline_coords).km
print(f"Distance to coastline: {distance_to_coastline:.2f} km")

# Draw the map with a PolyLine
# Coordinates for the PolyLine
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]

# Create the map centered at the launch site
map_launch_site = folium.Map(location=launch_site_coords, zoom_start=15)

# Add the PolyLine to the map
folium.PolyLine(locations=coordinates, weight=2, color='blue').add_to(map_launch_site)

# Add markers for the launch site and coastline
folium.Marker(location=launch_site_coords, popup="Launch Site").add_to(map_launch_site)
folium.Marker(location=coastline_coords, popup="Coastline").add_to(map_launch_site)

# Save the updated map
map_launch_site.save("task3_distance_to_coastline_map.html")
print("Task 3 map saved as 'task3_distance_to_coastline_map.html'")
# Display the map
map_launch_site

Distance to coastline: 0.51 km
Task 3 map saved as 'task3_distance_to_coastline_map.html'


## Interactive Map of Distance to Coastline:
- [Distance to Coastline](https://github.com/ShaWritesCode/MissionSpaceX_A-DataScience-Approach/blob/main/task3_distance_to_coastline_map.html)

In [249]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create and add a marker for the closest coastline
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline)
    )
)

# Add the marker to the map
distance_marker.add_to(site_map)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [251]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [
    [launch_site_lat, launch_site_lon],  # Launch site coordinates
    [coastline_lat, coastline_lon]       # Coastline coordinates
]

# Create a PolyLine between the launch site and the closest coastline
lines = folium.PolyLine(
    locations=coordinates,  # List of coordinates
    weight=2,               # Line thickness
    color='blue',           # Line color
    opacity=0.7            # Line opacity
)

# Add the PolyLine to the map
lines.add_to(site_map)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [261]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Example coordinates for a closest city, railway, or highway (replace with actual data)
city_lat = 28.604  # Example latitude for closest city
city_lon = -80.603  # Example longitude for closest city

# Calculate the distance between the launch site and the closest city/railway/highway
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a folium marker for the closest city/railway/highway
city_marker = folium.Marker(
    [city_lat, city_lon],
    popup="Closest City",
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#2980b9;"><b>{:10.2f} KM</b></div>'.format(distance_city)
    )
)

# Add the city marker to the map
city_marker.add_to(site_map)

# Create a PolyLine to draw a line between the launch site and the closest city
city_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]],  # Locations: launch site and city
    weight=2,                        # Line thickness
    color='green',                   # Line color
    opacity=0.7                      # Line opacity
)

# Add the PolyLine to the map
city_line.add_to(site_map)

# Save the updated map
site_map.save("task3_distance_to_closest_city_railway_highway_map.html")
print("Task 3 map saved as 'task3_distance_to_closest_city_railway_highway_map.html'")
# Display the map
site_map

Task 3 map saved as 'task3_distance_to_closest_city_railway_highway_map.html'


## Interactive Map of Distance to Closest City/Railway/Highway:
- [Distance Lines to the Proximities](https://github.com/ShaWritesCode/MissionSpaceX_A-DataScience-Approach/blob/main/task3_distance_to_closest_city_railway_highway_map.html)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
